In [ ]:
import torch 
from torchvision import ops

# apply roi pooling on proposals followed by avg pooling
roi_out = ops.roi_pool(feature_map, proposals_list, self.roi_size)
roi_out = self.avg_pool(roi_out)

image_path = "/mnt/hdd/eric/.tmp_ipy/15.Lab_Detection/00.Data/sample_images/000001.jpg"

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torchvision.ops import RoIPool
from matplotlib.patches import Rectangle


def visualize_tensor(image, bbox=None, object_num=0):

    image = image[object_num]  #
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])

    # tensor to img
    img_vis = np.array(image.permute(1, 2, 0), np.float32)  # C, H, W
    img_vis *= std
    img_vis += mean
    img_vis = np.clip(img_vis, 0, 1)

    plt.figure('input')
    plt.imshow(img_vis)

    if bbox is not None:
        bbox = bbox[0]
        for i in range(len(bbox)):

            x1 = bbox[i][0]
            y1 = bbox[i][1]
            x2 = bbox[i][2]
            y2 = bbox[i][3]

            # bounding box
            plt.gca().add_patch(Rectangle(xy=(x1, y1),
                                          width=x2 - x1,
                                          height=y2 - y1,
                                          linewidth=1,
                                          edgecolor=(0, 1, 0),
                                          facecolor='none'))

    plt.show()
    

if __name__ == '__main__':
    from PIL import Image
    import torchvision.transforms as tfs
    # 1. load image
    image = Image.open('./figures/000001.jpg').convert('RGB')
    # 2. transform image
    transforms = tfs.Compose([tfs.Resize((600, 600)),
                              tfs.ToTensor(),
                              tfs.Normalize(mean=[0.485, 0.456, 0.406],
                                            std=[0.229, 0.224, 0.225])])
    # 3. set image and boxes tensor
    image_tensor = transforms(image).unsqueeze(0)
    boxes_tensor = [torch.FloatTensor([[79.8867, 286.8000, 329.7450, 444.0000],
                                       [11.8980, 13.2000, 596.6006, 596.4000]])]
    boxes_tensor_scale_1 = [box_tensor/600 for box_tensor in boxes_tensor]

    # 4. roi pool for image
    roi_pool_for_image = RoIPool(output_size=(50, 50), spatial_scale=1.)
    roi_pool_for_image_scale_1 = RoIPool(output_size=(50, 50), spatial_scale=600.)
    results_roi_for_image = roi_pool_for_image(image_tensor, boxes_tensor)
    results_roi_for_image_scale_1 = roi_pool_for_image_scale_1(image_tensor, boxes_tensor_scale_1)
    print(results_roi_for_image.size())   # [num_object, 3, 50, 50]

    # 5. visualize
    visualize_tensor(image_tensor, boxes_tensor)
    visualize_tensor(results_roi_for_image, object_num=0)
    visualize_tensor(results_roi_for_image, object_num=1)
    visualize_tensor(results_roi_for_image_scale_1, object_num=0)
    visualize_tensor(results_roi_for_image_scale_1, object_num=1)

    # 6. roi pool for feature
    roi_pool_for_image = RoIPool(output_size=(7), spatial_scale=1.)
    print(roi_pool_for_image(image_tensor, boxes_tensor).size())